# Automatic DFA Learning

*April 2018 PMC@ISCAS*

---

![logo](http://iscasmc.ios.ac.cn/roll/lib/exe/fetch.php?media=wiki:logo.png)

This is a tutorial and demonstration of our Java Library [```Regular Omega Language Learning (ROLL)```](http://iscasmc.ios.ac.cn/roll/doku.php) in a Groovy kernel Jupyter Notebook.You do not need to worry if you use Java and not familiar with Groovy. Because the syntax of these two languages are similar.

In this notebook you will see 
- how to create a DFA as a learning target
- how to configure a learning process
- how to print the intermediate hypothesis and learner data structure in this hyper text environment

You can run this tutorial step by step, or you can run your code and see what will happend.

**Tips** : If something goes strange, just use the menu bar above ```Kernel -> Restart & Clear Output``` to rebot this notebook.

---

**First we load our compiled single fat jar.**

In [1]:
%classpath add jar ROLL.jar

Added jar: [ROLL.jar]


**Then we creat a target DFA on alphabet $\{a, b\}$ accepting $\{s \mid b \text{ occurs } 4 n + 3 \text{ times in } s \text{ (} n \ge 0 \text{)} \}$**

In [2]:
import roll.words.Alphabet
import roll.automata.DFA
import roll.automata.State

alphabet = new Alphabet();
alphabet.addLetter((char)'a');
alphabet.addLetter((char)'b');

// create an NBA with alphabet
target = new DFA(alphabet);
State fst = target.createState();
State snd = target.createState();
State thd = target.createState();
State fth = target.createState();


fst.addTransition(alphabet.indexOf((char)'a'), fst)
fst.addTransition(alphabet.indexOf((char)'b'), snd)
snd.addTransition(alphabet.indexOf((char)'a'), snd)
snd.addTransition(alphabet.indexOf((char)'b'), thd)
thd.addTransition(alphabet.indexOf((char)'a'), thd)
thd.addTransition(alphabet.indexOf((char)'b'), fth)
fth.addTransition(alphabet.indexOf((char)'a'), fth)
fth.addTransition(alphabet.indexOf((char)'b'), fst)

target.setInitial(fst)
target.setFinal(fth)

target

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 b <!-- 2->2 --> 2->2 a <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 b <!-- 3->0 --> 3->0 b <!-- 3->3 --> 3->3 a <!-- 4 --> 4 <!-- 4->0 --> 4->0

**Second we create an ```Options``` object to configure the learning process** 

For DFA we have 3 kinds of algorithm and 2 kinds data structure. But there are only 4 valid composition.

|                              | Options.Structure.TREE | Options.Structure.TABLE |
|:----------------------------:|:----------------------:|:-----------------------:|
| Options.Algorithm.DFA_LSTAR  | ✖️                     | ✔️                      |
| Options.Algorithm.DFA_KV     | ✔️                     | ✖️                      |
| Options.Algorithm.DFA_COLUMN | ✔️                     | ✔️                      |

Here we use a table based column algorithm as a demonstration.

In [3]:
import roll.main.Options  

options = new Options();
options.algorithm = Options.Algorithm.DFA_COLUMN;
options.structure = Options.Structure.TABLE;

options

LEARNING,TABLE,DFA_COLUMN,NONE,NBA,UNDER,verbose=false,bs=false,dot=false,inputfile=null,outputfile=null,outputA=null,outputB=null


**Then we start a Learning process**

```LearningSequence.ceList(target,options)``` takes two arguments. ```target``` is the target DFA we created which will be used as membership and equivalence oracle. ```options``` configures wichi type of learning algorithm will be used. And it returns an ```ImmutableList<QuerySimple>``` object contains all the counter examples used in the whole Automatic learning process.

In [4]:
import roll.main.*
celist = LearningSequence.ceList(target,options)

[bbb:ϵ, bbb:ϵ]

In [5]:
celist.get(0);

In [6]:
celist.get(1);

In [7]:
System.out.println(celist.getClass())
celist.size()

class com.google.common.collect.RegularImmutableList


2

There we can see that the table based column algorithm use two counter example to achieve the target DFA. Coincidentally, the two counter examples are both $bbb$.

And we can use ```LearningSequence.refinedLearner(target,options,celist)``` to get the intermediate learner object. Then we will see intermediate hypothesis and learner data. 

```LearningSequence.refinedLearner(target,options,celist)``` returns the learner refined by all the counter examples in ```celist```

In [8]:
import roll.main.*
finalLearner = LearningSequence.refinedLearner(target,options,celist)
System.out.println(finalLearner.getClass())
finalLearner

class roll.learner.dfa.table.LearnerDFATableColumn


     || ϵ | bb | b | 
ϵ    || - | -  | - | 
b    || - | +  | - | 
bb   || - | -  | + | 
bbb  || + | -  | - | 
a    || - | -  | - | 
ba   || - | +  | - | 
bba  || - | -  | + | 
bbba || + | -  | - | 
bbbb || - | -  | - | 


Now we can see the table used by the algorithm. For example, $b$ and $bb$ are in different equivalence class because the correspond rows are different.

```Learner::getHypothesis``` returns the current hypothesis DFA of a Learner.

In [9]:
import roll.main.*
LearningSequence.refinedLearner(target,options,celist.subList(0,0)).getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0

In [10]:
import roll.main.*
LearningSequence.refinedLearner(target,options,celist.subList(0,1)).getHypothesis()

Previous Hypothesis,Current Hypothesis
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?><!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 -->%3<!-- 0 -->00<!-- 0->0 -->0->0a<!-- 0->0 -->0->0b<!-- 1 -->1<!-- 1->0 -->1->0","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?><!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 -->%3<!-- 0 -->00<!-- 0->0 -->0->0a<!-- 1 -->11<!-- 0->1 -->0->1b<!-- 1->0 -->1->0b<!-- 1->1 -->1->1a<!-- 2 -->2<!-- 2->0 -->2->0"


We also show the preivous hypothesis DFA every time a Hypothesis is depicted on screen. **And the blue state was splited from the red one.**

In [11]:
import roll.main.*
LearningSequence.refinedLearner(target,options,celist.subList(0,2)).getHypothesis()

Previous Hypothesis 
 Current Hypothesis 
 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 <!-- 2->0 --> 2->0 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 b <!-- 2->2 --> 2->2 a <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 b <!-- 3->0 --> 3->0 b <!-- 3->3 --> 3->3 a <!-- 4 --> 4 <!-- 4->0 --> 4->0